In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
import os



In [2]:
train_data_dir = '/home/ravi/PROJECT_OM/dataset/train/'
validation_data_dir = '/home/ravi/PROJECT_OM/dataset/test/'


In [4]:
train_datagen = ImageDataGenerator(
                      rescale=1./255,
                      rotation_range=30,
                      shear_range=0.3,
                      zoom_range=0.3,
                      horizontal_flip=True,
                      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                         train_data_dir,
                         color_mode='grayscale',
                         target_size=(48,48),
                         batch_size=32,
                         class_mode='categorical',
                         shuffle=True)

Found 28709 images belonging to 7 classes.


In [5]:
validation_generator = validation_datagen.flow_from_directory(
                         validation_data_dir,
                         color_mode='grayscale',
                         target_size=(48,48),
                         batch_size=32,
                         class_mode='categorical',
                         shuffle=True)


Found 7178 images belonging to 7 classes.


In [6]:
class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise' ]
img, label = train_generator.__next__()


In [8]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(7, activation='softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 128)      0         
 2D)                                                  

In [14]:
train_path = '/home/ravi/PROJECT_OM/dataset/train/'
test_path = '/home/ravi/PROJECT_OM/dataset/test/'

In [15]:
num_train_imgs = 0
for root,dirs, files in os.walk(train_path):
    num_train_imgs += len(files)
    

In [16]:
num_test_imgs = 0
for root,dirs, files in os.walk(test_path):
    num_test_imgs += len(files)

In [17]:
print(num_test_imgs)
print(num_train_imgs)

7178
28709


In [18]:
epochs = 100
history = model.fit(train_generator,
                   steps_per_epoch = num_train_imgs//32,
                   epochs=epochs,
                   validation_data=validation_generator,
                   validation_steps=num_test_imgs//32)

Epoch 1/100
897/897 [==============================] - 25s 25ms/step - loss: 1.7857 - accuracy: 0.2602 - val_loss: 1.6912 - val_accuracy: 0.3238
Epoch 2/100
897/897 [==============================] - 28s 31ms/step - loss: 1.6668 - accuracy: 0.3315 - val_loss: 1.5132 - val_accuracy: 0.4171
Epoch 3/100
897/897 [==============================] - 28s 32ms/step - loss: 1.5274 - accuracy: 0.4038 - val_loss: 1.3629 - val_accuracy: 0.4754
Epoch 4/100
897/897 [==============================] - 27s 30ms/step - loss: 1.4448 - accuracy: 0.4442 - val_loss: 1.3440 - val_accuracy: 0.4856
Epoch 5/100
897/897 [==============================] - 28s 32ms/step - loss: 1.3855 - accuracy: 0.4681 - val_loss: 1.2456 - val_accuracy: 0.5247
Epoch 6/100
897/897 [==============================] - 22s 25ms/step - loss: 1.3454 - accuracy: 0.4844 - val_loss: 1.2126 - val_accuracy: 0.5329
Epoch 7/100
897/897 [==============================] - 25s 28ms/step - loss: 1.3144 - accuracy: 0.4979 - val_loss: 1.1969 - val_ac

In [20]:
model.save('model_file.h5')

In [2]:
import cv2
import numpy as np
from keras.models import load_model

In [3]:
model = load_model('model_file.h5')

In [4]:
faceDetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [23]:
# import urllib.request

# xml_url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"
# save_path = "haarcascade_frontalface_default.xml"

# # Download the XML file
# urllib.request.urlretrieve(xml_url, save_path)
# print("XML file downloaded successfully!")


XML file downloaded successfully!


In [5]:
labels_dict={0:'Angry',1:'Disgust',2:'Fear',3:'Happy',4:'Neutral',5:'Sad',6:'Surprise'}
# len(no. of images, image_height, image_width, channel)

In [13]:
frame=cv2.imread("image.png")
gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
faces=faceDetect.detectMultiScale(gray, 1.3, 3)

In [17]:
for x,y,w,h in faces:
    sub_face_img = gray[y:y+h, x:x+w]
    resized=cv2.resize(sub_face_img,(48,48))
    normalize=resized/255.0
    reshaped=np.reshape(normalize, (1,48,48,1))
    result=model.predict(reshaped)
    label=np.argmax(result, axis=1)[0]
    print(label)
    cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 1)
    cv2.rectangle(frame, (x,y), (x+w, y+h), (50,50,255), 2)
    cv2.rectangle(frame, (x,y-40), (x+w, y), (50,50,255), -1)
    cv2.putText(frame, labels_dict[label], (x,y-10), cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

6
1
4
3


In [ ]:
cv2.imshow("Frame",frame)
cv2.waitKey(0)
cv2.destroyAllWindows()